In [18]:
import numpy as np
import sklearn
import pandas as pd
import numpy as np
import nltk
import operator
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from sklearn.naive_bayes import GaussianNB

In [19]:
# read data from txt
def read_data(set_name):
    text_file_name  = set_name + "_text.txt"
    label_file_name = set_name + "_labels.txt"
    text_file = open("data/" + text_file_name, "r", encoding="utf8")
    label_file = open("data/" + label_file_name, "r", encoding="utf8")
    x = text_file.readlines()
    y = label_file.readlines()
    for i in range(len(y)): y[i] = y[i][0]
    return pd.DataFrame(x, columns=["text"]), pd.DataFrame(y, columns=["label"])
train_set_x, train_set_y = read_data("train")
val_set_x,   val_set_y   = read_data("val")
test_set_x,  test_set_y  = read_data("test")

In [20]:
def get_vector(vocab, text):
    vector = np.zeros(len(vocab))
    words = []
    for sentence in nltk.tokenize.sent_tokenize(text):
        for token in nltk.tokenize.word_tokenize(sentence):
            words.append(lemmatizer.lemmatize(token).lower())
    for i, word in enumerate(vocab):
        if word in words:
            vector[i] = words.count(word)
    return vector

In [21]:
# Label mapping declaration
SENTIMENTAL_MAP = {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
}

In [28]:
# Initial Feature: n most frequent words of each label class and combining them together
# define stopwords
stopwords = set(nltk.corpus.stopwords.words("english"))
additional_stopwords = [".", ",", "'s", "``", "''", "'", "n't", "%", "-", "$", "(", ")", ":", ";", "@", "&", "'m", "user", "#", "!", "?", "..."]
for sw in additional_stopwords: stopwords.add(sw)

# Initialize lemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()

# Get Vocabulary
vocabulary = []
n = 100
for label in SENTIMENTAL_MAP.keys():
    # get texts with same label
    temp_list = []
    for i in train_set_x.index:
        if train_set_y.loc[i, "label"] == label:
            temp_list.append(train_set_x.loc[i, "text"])
    
    # get n most frequent words of this label class
    dict_word_freq = {}
    for text in temp_list:
        for sentence in nltk.tokenize.sent_tokenize(text):
            for token in nltk.tokenize.word_tokenize(sentence):
                word = lemmatizer.lemmatize(token).lower()
                if word in stopwords: continue
                if word in dict_word_freq: dict_word_freq[word] += 1
                else: dict_word_freq[word] = 1
                
    # sort and add first n words in sorted list to vocabulary
    sorted_list = sorted(dict_word_freq.items(), key=operator.itemgetter(1), reverse=True)
    if n < len(sorted_list): sorted_list = sorted_list[:n]
    for word, frequency in sorted_list:
        if word not in vocabulary: vocabulary.append(word)

In [29]:
gnb = GaussianNB()
x, y = [], []
for i in train_set_x.index:
    x.append(get_vector(vocabulary, train_set_x.loc[i, "text"]))
    y.append(train_set_y.loc[i, "label"])

In [30]:
gnb.fit(np.asarray(x), np.asarray(y))

GaussianNB()

In [31]:
vx, vy = [], []
for i in val_set_x.index:
    vx.append(get_vector(vocabulary, val_set_x.loc[i, "text"]))
    vy.append(val_set_y.loc[i, "label"])
predictions = gnb.predict(vx)

In [32]:
print(str(precision_score(vy, predictions, average='macro')))
print(str(recall_score(vy, predictions, average='macro')))
print(str(f1_score(vy, predictions, average='macro')))
print(str(accuracy_score(vy, predictions)))

0.5000896186203714
0.5005342758507315
0.49679783626740553
0.531
